In [2]:
import pandas as pd
import numpy as np

In [3]:
# Objectif : df avec un film par ligne et le maximum d'infos 
# Etape 1 : Merge de toutes les tables hors film 
# Etape 2 : Rassenblement des colonne similaire 
# Etape 3 : Remplissage des infos manquantes ou suppression des lignes 



In [4]:
# Etape 1 : Merge de toutes les tables hors film 
# Pour la table title akas ont supprime directement les film qui ne sont pas en français (un cinema ne peut pas proposer de film non traduit)
# Il y a aussi des doublons de lignes 
df_title_akas = pd.read_csv("./BD/title.akas.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
df_title_akas = df_title_akas[(df_title_akas['region'] == 'FR') | (df_title_akas['language'] == 'fr')]
df_title_akas = df_title_akas.drop_duplicates(subset='titleId', keep='first')

# Pour la table title basics on ne récupère que les films (titleType == 'movie')
df_title_basics = pd.read_csv("BD/title.basics.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
df_title_basics = df_title_basics[(df_title_basics['titleType'] == 'movie')]

# On ne s'occupe pas de title crew pour le moment car on s'en occupera dans la gestion des acteurs
# On ne s'occupe pas de title episode car elle sert pour les series (vu dans le doc) 
# On ne s'occupe pas de title principals car on s'en occupera dans la gestion des acteurs 

# Pour la table title rating on récupère tout mais on attend le même nombre que précédement 
df_title_rating = pd.read_csv("BD/title.ratings.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')

# On ne s'occupe pas de name basics car on s'en occupera dans la gestion des acteurs 

# Pour la table TMBD FULL on récupère tout 
df_tmdb_full = pd.read_csv("./BD/tmdb_full.csv")
df_tmdb_full['fr?'] = df_tmdb_full['spoken_languages'].apply(lambda x: 'fr' in x)
df_tmdb_full = df_tmdb_full[df_tmdb_full['fr?']]

C:\Users\lepor\AppData\Local\Temp\ipykernel_55644\527731522.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv("./BD/title.akas.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
C:\Users\lepor\AppData\Local\Temp\ipykernel_55644\527731522.py:9: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv("BD/title.basics.tsv.gz", compression = 'gzip', na_values=['\\N'], sep = '\t')
C:\Users\lepor\AppData\Local\Temp\ipykernel_55644\527731522.py:22: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb_full = pd.read_csv("./BD/tmdb_full.csv")


In [5]:
# Merge entre df_title_akas et df_title_basics (attendu df_merge1 >= df_title_akas ou df_title_basics)
df_merge1 = pd.merge(df_title_akas,
                     df_title_basics,
                     how = 'right',
                     left_on = df_title_akas['titleId'],
                     right_on = df_title_basics['tconst'])
df_merge1['tconst'] = df_merge1['key_0']
df_merge1 = df_merge1.drop(['key_0', 'titleId'], axis = 1)

In [6]:
# Merge entre df_merge1 et df_title_rating (attendu df_merge2 = df_merge1 (en ligne))
df_merge2 = pd.merge(df_merge1,
                     df_title_rating,
                     how = 'left',
                     on = 'tconst')

In [7]:
# Merge entre df_merge2 et df_tmdb_full (attendu df_merge3 >= df_merge1 ou df_tmdb_full (en ligne))
df_merge3 = pd.merge(df_merge2,
                     df_tmdb_full,
                     how = 'outer',
                     left_on = df_merge2['tconst'],
                     right_on = df_tmdb_full['imdb_id'])
df_merge3

,key_0,ordering,title_x,region,language,types,attributes,isOriginalTitle,tconst,titleType,...,spoken_languages,status,tagline,title_y,video,vote_average,vote_count,production_companies_name,production_companies_country,fr?
0,tt0000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,['fr'],Released,NaN,Poor Pierrot,False,5.994,82.0,[],NaN,True
1,tt0000009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0000009,movie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0000147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0000147,movie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0000306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"['fr', 'xx']",Released,NaN,Le duel d'Hamlet,False,5.100,9.0,['Phono-Cinéma-Théâtre'],NaN,True
4,tt0000502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0000502,movie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703132,tt9916622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt9916622,movie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703133,tt9916680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt9916680,movie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703134,tt9916706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt9916706,movie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703135,tt9916730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Etape 2 : Rassenblement des colonne similaire (lors d'un choix on priorise arbitrairement la table TMBD)
# tconst / imdb_id / id sont à retirer
# On le titre title_y / title_x / primaryTitle / originalTitle
# Le genre est à gérer genres_x / genres_y
# La note averageRating et vote_average
# Le nombre de vote vote_count et numVotes

In [9]:
# tconst / imdb_id / id sont à retirer
df_merge3['tconst'] = df_merge3['key_0']
df_merge3 = df_merge3.drop(['key_0', 'imdb_id', 'id'], axis = 1)

In [10]:
# On le titre title_y / title_x / primaryTitle / originalTitle
# Créer une nouvelle colonne 'titre' en priorisant les colonnes dans l'ordre
df_merge3['titre'] = df_merge3['title_y'].fillna(df_merge3['title_x']).fillna(df_merge3['primaryTitle']).fillna(df_merge3['originalTitle'])
df_merge3 = df_merge3.drop(['title_y', 'title_x', 'primaryTitle', 'originalTitle'], axis = 1)


In [11]:
# Le genre est à gérer genres_x / genres_y
df_merge3['genres_y'] = df_merge3['genres_y'].apply(lambda x: x if (type(x) == list) else [])
df_merge3['genres_x_list'] = df_merge3['genres_x'].fillna('').str.split(', ')
df_merge3['genre'] = df_merge3.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)
df_merge3 = df_merge3.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)
df_merge3['genre'] = df_merge3['genre'].apply(lambda x: x if x != [] else np.nan)


In [12]:
# La note averageRating et vote_average et le nombre de vote vote_count et numVotes
df_merge3['averageRating'] = df_merge3['averageRating'].fillna(0)
df_merge3['numVotes'] = df_merge3['numVotes'].fillna(0)
df_merge3['vote_average'] = df_merge3['vote_average'].fillna(0)
df_merge3['vote_count'] = df_merge3['vote_count'].fillna(0)
df_merge3['Rate'] = ((df_merge3['averageRating'] * df_merge3['numVotes'] + df_merge3['vote_average'] * df_merge3['vote_count']) / (df_merge3['numVotes'] + df_merge3['vote_count']))
df_merge3['NbRate'] = df_merge3['numVotes'] + df_merge3['vote_count']
df_merge3.drop(['averageRating', 'numVotes', 'vote_count', 'vote_average'], axis=1, inplace=True)


In [ ]:
# La -18 à fusionner entre isAdult (0 ou 1) et adult (bool)

In [ ]:
# release_date et startYear à fusuionner 

In [ ]:
# runtime et runtimeMinutes à fusionner 

In [ ]:
# tagline et overview à fusionner ???

In [ ]:
# Les films dont le status est cancelled à retirer 

In [ ]:
# 

In [13]:
df_merge3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703137 entries, 0 to 703136
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   ordering                      99153 non-null   float64
 1   region                        99153 non-null   object 
 2   language                      32823 non-null   object 
 3   types                         80256 non-null   object 
 4   attributes                    2491 non-null    object 
 5   isOriginalTitle               99153 non-null   float64
 6   tconst                        703137 non-null  object 
 7   titleType                     697175 non-null  object 
 8   isAdult                       697175 non-null  float64
 9   startYear                     595820 non-null  float64
 10  endYear                       0 non-null       float64
 11  runtimeMinutes                438942 non-null  object 
 12  adult                         20689 non-null

In [ ]:
df = df_merge3.drop(['language', 'types', 'attributes', 'titleType', 'endYear', 'original_title', 'spoken_languages', 'video', 'fr?', 'titleType'], axis = 1)

array([nan, 'reissue title', 'new title', 'poster title',
       'complete title', 'DVD box title', 'short title', 'IMAX version',
       'restored version', 'first part title', 'pre-release title',
       'second part title', 'Cannes festival title', 'dubbed version',
       'silent version', 'theatrical title',
       'alternative transliteration', 'literal title',
       'alternative spelling', 'informal title', 'cable TV title',
       '16mm release title', 'first episode title', 'subtitle',
       'rerun title', 'video box title', 'long title', 'short version',
       'informal alternative title', 'transliterated title',
       'informal literal title', 'review title',
       'orthographically correct title', 'trailer title',
       'original subtitled version', '8mm release title',
       'uncensored intended title', 'cut version',
       'literal English title', 'literal translation of working title',
       'segment title', 'informal short title', 'censored version',
       're

In [28]:
df_merge3['production_companies_country'].unique()

array([nan, "['FR']", "['']", ..., "['FR', 'LU', 'BE', 'BE', 'BE', 'BE']",
       "['', 'MX', 'MX', '', '', 'MX', '']", "['JP', 'FR', 'FR']"],
      dtype=object)

In [16]:
df_merge3['types'].unique()

array([nan, 'imdbDisplay', 'tv', 'alternative', 'video', 'dvd', 'working',
       'festival'], dtype=object)